# Import Libraries

In [29]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [30]:
# Install tmdbsimple (only need to run once)
!pip install tqdm

# Load Credentials

In [31]:
# Load API Credentials
with open('/Users/tjbingamon/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [32]:
# set our API call parameters 
LOCATION = 'Portland, OR'
TERM = 'Pizza'

In [33]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_PDX_pizza.json"
JSON_FILE

'Data/results_in_progress_PDX_pizza.json'

# Check JSON File

In [34]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_PDX_pizza.json not found. Saving empty list to file.


In [35]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


# Yelp API

In [36]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [37]:
## How many results total?
total_results = results['total']
total_results

1700

In [38]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [39]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

85

In [40]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [41]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/85 [00:00<?, ?it/s]

In [42]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [43]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_PDX_pizza.json already exists. Deleting previous file...
[i] Data/results_in_progress_PDX_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


85

In [45]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/85 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


# Display Results

In [46]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,-loAMvamhl-6kGw9jnetww,lovelys-fifty-fifty-portland,Lovely's Fifty Fifty,https://s3-media2.fl.yelpcdn.com/bphoto/bjjx_D...,False,https://www.yelp.com/biz/lovelys-fifty-fifty-p...,463,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 45.5528680273715, 'longitude': -1...",[delivery],$$,"{'address1': '4039 Mississippi Ave N', 'addres...",+15032814060,(503) 281-4060,2020.077922
1,ywxapaOnDn3wyof-35BUIQ,life-of-pie-pizza-portland,Life of Pie Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/LhGaAT...,False,https://www.yelp.com/biz/life-of-pie-pizza-por...,864,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 45.549394, 'longitude': -122.666518}",[delivery],$$,"{'address1': '3632 N Williams Ave', 'address2'...",+15038200083,(503) 820-0083,1217.933062
2,48-tBsPQlSw5VD0IX6CxSg,baby-doll-pizza-portland,Baby Doll Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/C9t72N...,False,https://www.yelp.com/biz/baby-doll-pizza-portl...,517,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 45.5195313, 'longitude': -122.636...",[delivery],$$,"{'address1': '2835 SE Stark St', 'address2': '...",+15034594450,(503) 459-4450,2922.721313
3,lBRlHsXgBwWszAaVcV6dYg,sizzle-pie-portland,Sizzle Pie,https://s3-media2.fl.yelpcdn.com/bphoto/2d8twF...,False,https://www.yelp.com/biz/sizzle-pie-portland?a...,1102,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 45.5227460279759, 'longitude': -1...","[delivery, pickup]",$,"{'address1': '624 E Burnside', 'address2': '',...",+15032347437,(503) 234-7437,2254.819850
4,Nr0cDHVHAgfGqrqw5x8ZaA,pizza-thief-portland,Pizza Thief,https://s3-media1.fl.yelpcdn.com/bphoto/vMQzuu...,False,https://www.yelp.com/biz/pizza-thief-portland?...,140,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 45.53659869202158, 'longitude': -...",[delivery],$$,"{'address1': '2610 NW Vaughn St', 'address2': ...",+15037197778,(503) 719-7778,4042.713209


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,2vqfrDOCEGrtTPFQfn0MGQ,trader-joes-portland,Trader Joe's,https://s3-media1.fl.yelpcdn.com/bphoto/DYdrlt...,False,https://www.yelp.com/biz/trader-joes-portland?...,172,"[{'alias': 'grocery', 'title': 'Grocery'}]",4.0,"{'latitude': 45.5344, 'longitude': -122.6208}",[],$$,"{'address1': '4121 NE Halsey St', 'address2': ...",+15032844232,(503) 284-4232,2741.974846
996,vn044qnNg9OuGs_NFvm7NQ,dominos-pizza-tualatin,Domino's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/GYdnvd...,False,https://www.yelp.com/biz/dominos-pizza-tualati...,22,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 45.38207, 'longitude': -122.74803}",[],$,"{'address1': '7066 SW Nyberg St', 'address2': ...",+15037838100,(503) 783-8100,19283.598038
997,4fqP6o_iSlyHy-WodU2kbA,new-happy-fortune-chinese-restaurant-portland,New Happy Fortune Chinese Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/lDLZTp...,False,https://www.yelp.com/biz/new-happy-fortune-chi...,127,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.5,"{'latitude': 45.53815, 'longitude': -122.57925}","[delivery, pickup]",$,"{'address1': '2145 NE 82nd Ave', 'address2': '...",+15032531647,(503) 253-1647,5854.213493
998,k9e2vDPLYrP1wo7pBx9cxQ,ruddick-wood-newberg,Ruddick/Wood,https://s3-media3.fl.yelpcdn.com/bphoto/EGLylN...,False,https://www.yelp.com/biz/ruddick-wood-newberg?...,428,"[{'alias': 'newamerican', 'title': 'American (...",3.5,"{'latitude': 45.30017855424348, 'longitude': -...",[],$$,"{'address1': '720 E 1st St', 'address2': '', '...",+15034876133,(503) 487-6133,36635.649777
999,QoDoOB3S1mK0qtqOmo8kSQ,mcmenamins-edgefield-troutdale-2,McMenamins Edgefield,https://s3-media2.fl.yelpcdn.com/bphoto/CS4UEA...,False,https://www.yelp.com/biz/mcmenamins-edgefield-...,1436,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali...",4.0,"{'latitude': 45.5374175833515, 'longitude': -1...",[],$$,"{'address1': '2126 SW Halsey St.', 'address2':...",+15036698610,(503) 669-8610,19365.803664


In [47]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [49]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)